In [2]:
import pdfplumber
import pytesseract
from pytesseract import Output
from PIL import Image

# Path to the scanned PDF
pdf_path = r"C:\Users\spolo\OneDrive\Documents\DOC\Work\TCS\Code\dynamic_data_masking\presidio_ddm\pdf_files\sample_sensitive.pdf"

# Open the PDF with pdfplumber
with pdfplumber.open(pdf_path) as pdf:
    for i, page in enumerate(pdf.pages):
        image = page.to_image(resolution=500).original
        page_number = i + 1
        words_info = []
        # img = Image.open(image)
        custom_confg = r'--oem 3 --psm 6'
        ocr_data = pytesseract.image_to_data(image, output_type=Output.DICT, lang='eng')
        text = pytesseract.image_to_string(image, config=custom_confg, lang='eng')
        word_info = []
        for i in range(len(ocr_data['text'])):
            word_info = {
                'text': ocr_data['text'][i],
                'start_x': ocr_data['left'][i] * (page.width / image.width),
                'start_y': ocr_data['top'][i] * (page.height / image.height),
                'end_x': (ocr_data['left'][i] + ocr_data['width'][i]) * (page.width / image.width),
                'end_y': (ocr_data['top'][i] + ocr_data['height'][i]) * (page.height / image.height),
                'page_number': page_number
            }
            words_info.append(word_info)

In [14]:
print(text)

Hello, my name is John Doe.

You can reach me at john.doe@example.com or at my phone number +1-555-123-4567.

My credit card number is 4111-1111-1111-1111 and my home address is 1234 Elm Street,
Springfield.

| have diabetes, and asthma. | vote for Democratic party and | am ecologist.

Best regards, John.



# ONE MAIN FILE

In [59]:
from presidio_analyzer import AnalyzerEngineProvider
from presidio_anonymizer import AnonymizerEngine

# Path to your configuration file
conf_path = r'C:\Users\spolo\OneDrive\Documents\DOC\Work\TCS\Code\dynamic_data_masking\presidio_ddm\dynamic_data_masking\ddm_config\analyzer_config\all-config-only-C4.yaml'

# Initialize the AnalyzerEngineProvider with your configuration file
provider = AnalyzerEngineProvider(analyzer_engine_conf_file=conf_path)
analyzer = provider.create_engine()

# Input text
text = """
Hello, Bonjour,  my name is John Doe.
You can reach me at john.doe@example.com or at my phone number +1-555-123-4567.
My credit card number is 4111-1111-1111-1111 and my home address is 1234 Elm Street, Springfield.
I have diabetes, and asthma. I vote for the Democratic party and I am an ecologist.
Best regards, John.
"""

# Analyze the text
analyzer_result = analyzer.analyze(text=text, language='fr')

# Filter results to include only custom entities (e.g., GREET and HEALTH)
# custom_entities = ["GREET", "HEALTH"]
# filtered_results = [result for result in analyzer_result if result.entity_type in custom_entities]

# Anonymize the text using only the filtered results
anonimyzer = AnonymizerEngine()
result = anonimyzer.anonymize(text=text, analyzer_results=analyzer_result)

# Print the anonymized text
print(result.text)


Hello, <GREET>,  my name is John Doe.
You can reach me at john.doe@example.com or at my phone number +1-555-123-4567.
My credit card number is 4111-1111-1111-1111 and my home address is 1234 Elm Street, Springfield.
I have diabetes, and asthma. I vote for the Democratic party and I am an ecologist.
Best regards, John.



# 3 FILES

In [56]:
from presidio_analyzer import AnalyzerEngine, AnalyzerEngineProvider

text = """
Hello, Bonjour,  my name is John Doe.
You can reach me at john.doe@example.com or at my phone number +1-555-123-4567.
My credit card number is 4111-1111-1111-1111 and my home address is 1234 Elm Street, Springfield.
I have diabetes, and asthma. I vote for the Democratic party and I am an ecologist.
Best regards, John.
"""
analyzer_conf_file = r"C:\Users\spolo\OneDrive\Documents\DOC\Work\TCS\Code\dynamic_data_masking\presidio_ddm\dynamic_data_masking\ddm_config\analyzer_config\fr-analyzer-config-c4.yaml"
nlp_engine_conf_file = r"C:\Users\spolo\OneDrive\Documents\DOC\Work\TCS\Code\dynamic_data_masking\presidio_ddm\dynamic_data_masking\ddm_config\analyzer_config\fr-nlp-config.yaml"
# recognizer_registry_conf_file = r"C:\Users\spolo\OneDrive\Documents\DOC\Work\TCS\Code\dynamic_data_masking\presidio_ddm\dynamic_data_masking\ddm_config\analyzer_config\fr-recognizers-config.yaml"

provider = AnalyzerEngineProvider(
    analyzer_engine_conf_file=analyzer_conf_file,
    # nlp_engine_conf_file=nlp_engine_conf_file,
    # recognizer_registry_conf_file=recognizer_registry_conf_file,
    )
analyzer = provider.create_engine()

results = analyzer.analyze(text="My name is Morris", language="fr")
print(results)

KeyError: 'fr'

In [7]:
from collections import defaultdict
original_words = set(text.split())
new_words = set(result.text.split())
differing_words = original_words.difference(new_words)
word_data_map = defaultdict(list)
for word_data in words_info:
    word_data_map[word_data['text']].append(word_data)
differing_words_data = [data for word in differing_words for data in word_data_map[word]]



In [8]:
print(text)

Hello, my name is John Doe.

You can reach me at john.doe@example.com or at my phone number +1-555-123-4567.

My credit card number is 4111-1111-1111-1111 and my home address is 1234 Elm Street,
Springfield.

| have diabetes, and asthma. | vote for Democratic party and | am ecologist.

Best regards, John.



In [9]:
print(result.text)

<GREET>, my name is <PERSON>.

You can reach me at john.doe@example.com or at my phone number <NRP>-123-4567.

My credit card number is <DATE_TIME> and my home address is <DATE_TIME> Elm Street,
<LOCATION>.

| have diabetes, and asthma. | vote for <NRP> party and | am ecologist.

Best regards, <PERSON>.



In [10]:
words_info

[{'text': '',
  'start_x': 0.0,
  'start_y': 0.0,
  'end_x': 595.28,
  'end_y': 841.89,
  'page_number': 1},
 {'text': '',
  'start_x': 31.24716013546202,
  'start_y': 37.43653155464341,
  'end_x': 563.0248669569424,
  'end_y': 161.9849923037455,
  'page_number': 1},
 {'text': '',
  'start_x': 32.11113691340106,
  'start_y': 37.43653155464341,
  'end_x': 183.59506531204644,
  'end_y': 48.667491021036426,
  'page_number': 1},
 {'text': '',
  'start_x': 32.11113691340106,
  'start_y': 37.43653155464341,
  'end_x': 183.59506531204644,
  'end_y': 48.667491021036426,
  'page_number': 1},
 {'text': 'Hello,',
  'start_x': 32.11113691340106,
  'start_y': 37.43653155464341,
  'end_x': 60.76636671504596,
  'end_y': 47.803571062083115,
  'page_number': 1},
 {'text': 'my',
  'start_x': 65.95022738268021,
  'start_y': 39.74031811185223,
  'end_x': 81.06982099661344,
  'end_y': 48.667491021036426,
  'page_number': 1},
 {'text': 'name',
  'start_x': 85.38970488630866,
  'start_y': 39.74031811185223,


In [11]:
print(result.text)

<GREET>, my name is <PERSON>.

You can reach me at john.doe@example.com or at my phone number <NRP>-123-4567.

My credit card number is <DATE_TIME> and my home address is <DATE_TIME> Elm Street,
<LOCATION>.

| have diabetes, and asthma. | vote for <NRP> party and | am ecologist.

Best regards, <PERSON>.



In [12]:
pdf_path = r"C:\Users\spolo\OneDrive\Documents\DOC\Work\TCS\Code\dynamic_data_masking\ddm\pdf_files\sample_sensitive_pdf.pdf"

output_pdf_path = r"C:\Users\spolo\OneDrive\Documents\DOC\Work\TCS\Code\dynamic_data_masking\ddm\pdf_files\sample_sensitive_pdf_redacted.pdf"

from reportlab.pdfgen import canvas
from PyPDF2 import PdfWriter, PdfReader
temp_pdf_path = "temp_overlay.pdf"
# Step 1: Create a canvas for the overlay PDF
with pdfplumber.open(pdf_path) as pdf:
    overlay = canvas.Canvas(temp_pdf_path)

    for page_num, page in enumerate(pdf.pages, start=1):
        page_width = page.width
        page_height = page.height
        overlay.setPageSize((page_width, page_height))

        page_words = [word for word in differing_words_data if word['page_number'] == page_num]
        for word_data in page_words:
            x0, y0, x1, y1 = word_data['start_x'], word_data['start_y'], word_data['end_x'], word_data['end_y']
            overlay.setFillColor('black')
            overlay.setStrokeColor('black')
            overlay.setLineWidth(0.5)
            overlay.rect(x0, page_height - y1, x1 - x0, y1 - y0, fill=1)

        overlay.showPage()

    overlay.save()

input_pdf = PdfReader(pdf_path)
overlay_pdf = PdfReader(temp_pdf_path)
writer = PdfWriter()

for i, page in enumerate(input_pdf.pages):
    if i < len(overlay_pdf.pages):
        page.merge_page(overlay_pdf.pages[i])
    writer.add_page(page)
    
with open(output_pdf_path, "wb") as output_file:
    writer.write(output_file)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\spolo\\OneDrive\\Documents\\DOC\\Work\\TCS\\Code\\dynamic_data_masking\\ddm\\pdf_files\\sample_sensitive_pdf.pdf'